In [4]:
# !pip install -r requirements.txt
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [5]:
# DB 클라이언트 import
from db_client import RDSClient

import pandas as pd
# DB 연결
db = RDSClient()

DB HOST: musinsa-db.c07kuo6ug98z.us-east-1.rds.amazonaws.com
DB USER: admin
DB PASSWORD: qkqajrwkrnrnrn9_
DB NAME: musinsa
DB PORT: 3306
✅ DB Engine (Pool) 생성 완료


In [6]:
query5 = """
    SELECT
        ps.product_id,
        ps.product_name,
        ps.upper_category,
        ps.img_url
    FROM products ps
"""
result2 = db.execute(query5)
products_df = pd.DataFrame(result2)

In [7]:
products_df

,product_id,product_name,upper_category,img_url
0,47144,DVS PIPING WING TIP,신발,https://image.msscdn.net/images/goods_img/2013...
1,52661,베이직 유광소가죽 페니로퍼 4701 [BLACK BOX],신발,https://image.msscdn.net/images/goods_img/2013...
2,52662,베이직 유광소가죽 페니로퍼 4701 [BROWN ADVAN],신발,https://image.msscdn.net/images/goods_img/2013...
3,54267,사이드라인 스탠다드 테슬로퍼 6300 [WINE BROWN],신발,https://image.msscdn.net/images/goods_img/2013...
4,54268,사이드라인 스탠다드 테슬로퍼 6300 [OIL BROWN],신발,https://image.msscdn.net/images/goods_img/2013...
...,...,...,...,...
200085,5814983,코펙 GTX 남성,신발,https://image.msscdn.net/images/goods_img/2025...
200086,5815004,노반 4 니발리스 남성,신발,https://image.msscdn.net/images/goods_img/2025...
200087,5815170,성인 니트 스킨슈즈 화이트 RO-RS2D001-WH,신발,https://image.msscdn.net/images/goods_img/2025...
200088,5815171,성인 니트 스킨슈즈 블랙 RO-RS2D001-BK,신발,https://image.msscdn.net/images/goods_img/2025...


In [8]:
sample_products = (
    products_df
    .groupby('upper_category', group_keys=False)
    .apply(lambda x: x.sample(n=5, random_state=42))
    .reset_index(drop=True)
)
sample_products

ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
# ===================== 누끼 라이브러리 rembg =======================
!pip install rembg pillow requests

In [ ]:
sample_products

In [7]:
# JSON 파일로 저장
sample_products.to_json(
    "sample_products.json",
    orient="records",
    force_ascii=False,
    indent=2
)

In [ ]:

# DB 연결
db = RDSClient()

query2 = """
    SELECT
        pi.product_id,
        pi.brand,
        pi.img_url
    FROM product_top pi
"""

# 쿼리 실행 및 DataFrame 변환
result = db.execute(query2)
img_df = pd.DataFrame(result)
print(f"데이터베이스에서 {len(img_df)}개의 상품 정보를 불러왔습니다.")

import torch
import open_clip
import numpy as np
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import pickle

device = "cuda" if torch.cuda.is_available() else "cpu"

model, preprocess, tokenizer = open_clip.create_model_and_transforms(
    'ViT-B-32',
    pretrained='laion2b_s34b_b79k'
)
model = model.to(device)
model.eval()

# 이미지 URL 로딩 함수
def load_image_from_url(url, timeout=2):
    try:
        r = requests.get(url, timeout=timeout)
        r.raise_for_status()
        img = Image.open(BytesIO(r.content)).convert("RGB")
        return img
    except:
        return None

batch_size = 128

image_embeddings = []
valid_product_ids = []

batch_imgs = []
batch_ids = []

print(f"총 {len(img_df)}개의 상품 이미지를 처리합니다...")

for _, row in tqdm(img_df.iterrows(), total=len(img_df), desc="이미지 임베딩 중"):
    img = load_image_from_url(row['img_url'])
    if img is None:
        continue

    batch_imgs.append(preprocess(img))
    batch_ids.append(row['product_id'])

    if len(batch_imgs) == batch_size:
        batch = torch.stack(batch_imgs).to(device)
        with torch.no_grad():
            emb = model.encode_image(batch)
            emb = emb / emb.norm(dim=-1, keepdim=True)

        image_embeddings.append(emb.cpu().numpy())
        valid_product_ids.extend(batch_ids)

        batch_imgs, batch_ids = [], []

# 마지막 배치 처리
if batch_imgs:
    batch = torch.stack(batch_imgs).to(device)
    with torch.no_grad():
        emb = model.encode_image(batch)
        emb = emb / emb.norm(dim=-1, keepdim=True)

    image_embeddings.append(emb.cpu().numpy())
    valid_product_ids.extend(batch_ids)

# 임베딩 벡터 결합
image_embeddings = np.vstack(image_embeddings)

print(f"\n임베딩 완료: {len(valid_product_ids)}개 상품")
print(f"임베딩 shape: {image_embeddings.shape}")

# 파일로 저장
print("\n파일 저장 중...")

# 1. NumPy 형식으로 저장 (.npz - 압축)
np.savez_compressed(
    'product_image_embeddings.npz',
    embeddings=image_embeddings,
    product_ids=np.array(valid_product_ids)
)
print("✓ product_image_embeddings.npz 저장 완료")

# 2. 개별 파일로도 저장 (선택사항)
np.save('image_embeddings.npy', image_embeddings)
np.save('product_ids.npy', np.array(valid_product_ids))
print("✓ image_embeddings.npy, product_ids.npy 저장 완료")

# 3. Pickle 형식으로도 저장 (딕셔너리 형태)
embedding_data = {
    'embeddings': image_embeddings,
    'product_ids': valid_product_ids,
    'embedding_dim': image_embeddings.shape[1],
    'model_name': 'ViT-B-32',
    'pretrained': 'laion2b_s34b_b79k'
}

with open('product_embeddings.pkl', 'wb') as f:
    pickle.dump(embedding_data, f)
print("✓ product_embeddings.pkl 저장 완료")

print(f"\n저장 완료! 총 {len(valid_product_ids)}개 상품의 임베딩이 저장되었습니다.")

# 저장된 파일 로드 예시
print("\n=== 파일 로드 예시 ===")
print("""
# NPZ 파일 로드
data = np.load('product_image_embeddings.npz')
embeddings = data['embeddings']
product_ids = data['product_ids']

# 또는 개별 파일 로드
embeddings = np.load('image_embeddings.npy')
product_ids = np.load('product_ids.npy')

# Pickle 파일 로드
with open('product_embeddings.pkl', 'rb') as f:
    data = pickle.load(f)
    embeddings = data['embeddings']
    product_ids = data['product_ids']
""")